In [1]:
# !pip install catboost==0.15.2
# !pip install tqdm
# !pip install xlrd
import numpy as np
import pandas as pd
import catboost as cbt
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
import gc
import math
import time
from tqdm import tqdm
import datetime
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train = pd.read_csv('.\\data\\first_round_training_data.csv')
test = pd.read_csv('.\\data\\first_round_testing_data.csv')
submit = pd.read_csv('.\\data\\submit_example.csv')
data = train.append(test).reset_index(drop=True)
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
data['label'] = data['Quality_label'].map(dit)
train['label'] = train['Quality_label'].map(dit)

In [5]:
feature_name = ['Parameter{0}'.format(i) for i in range(5, 11)]
tr_index = ~data['label'].isnull()
X_train = data[tr_index][feature_name].reset_index(drop=True)
y = data[tr_index]['label'].reset_index(drop=True).astype(int)
X_test = data[~tr_index][feature_name].reset_index(drop=True)

print(X_train.shape,X_test.shape)
oof = np.zeros((X_train.shape[0],4))
prediction = np.zeros((X_test.shape[0],4))
seeds = [1, 2, 3, 4, 5, 6]
num_model_seed = 6
for model_seed in range(num_model_seed):
    print("out loop", model_seed + 1)
    oof_cat = np.zeros((X_train.shape[0],4))
    prediction_cat=np.zeros((X_test.shape[0],4))
    skf = StratifiedKFold(n_splits=5, random_state=seeds[model_seed], shuffle=True)
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        print("inner loop", index)
        train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        gc.collect()
        cbt_model = cbt.CatBoostClassifier(iterations=800,learning_rate=0.01,verbose=300,
                                        early_stopping_rounds=200,task_type='GPU',
                                        loss_function='MultiClass')
        cbt_model.fit(train_x, train_y ,eval_set=(train_x, train_y))
        oof_cat[test_index] += cbt_model.predict_proba(test_x)
        prediction_cat += cbt_model.predict_proba(X_test)/5
        gc.collect()
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
    print('logloss',log_loss(pd.get_dummies(y).values, oof_cat))
    print('ac',accuracy_score(y, np.argmax(oof_cat,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y] - oof_cat))/480))
print('logloss',log_loss(pd.get_dummies(y).values, oof))
print('ac',accuracy_score(y, np.argmax(oof,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[y] - oof))/480))

(6000, 6) (6000, 6)
out loop 1
inner loop 0
0:	learn: 1.3808195	test: 1.3808196	best: 1.3808196 (0)	total: 73.6ms	remaining: 58.8s
300:	learn: 1.0816335	test: 1.0816335	best: 1.0816335 (300)	total: 7.11s	remaining: 11.8s
600:	learn: 1.0476845	test: 1.0476844	best: 1.0476844 (600)	total: 14.1s	remaining: 4.66s
799:	learn: 1.0319359	test: 1.0319359	best: 1.0319359 (799)	total: 18.4s	remaining: 0us
bestTest = 1.031935914
bestIteration = 799
Shrink model to first 800 iterations.
inner loop 1
0:	learn: 1.3808341	test: 1.3808343	best: 1.3808343 (0)	total: 24.7ms	remaining: 19.7s
300:	learn: 1.0844651	test: 1.0844651	best: 1.0844651 (300)	total: 7.03s	remaining: 11.6s
600:	learn: 1.0514470	test: 1.0514471	best: 1.0514471 (600)	total: 13.9s	remaining: 4.61s
799:	learn: 1.0361601	test: 1.0361601	best: 1.0361601 (799)	total: 18.4s	remaining: 0us
bestTest = 1.036160086
bestIteration = 799
Shrink model to first 800 iterations.
inner loop 2
0:	learn: 1.3812908	test: 1.3812905	best: 1.3812905 (0)	to

bestTest = 1.03909891
bestIteration = 799
Shrink model to first 800 iterations.
inner loop 3
0:	learn: 1.3808540	test: 1.3808539	best: 1.3808539 (0)	total: 16.9ms	remaining: 13.5s
300:	learn: 1.0862226	test: 1.0862225	best: 1.0862225 (300)	total: 8.68s	remaining: 14.4s
600:	learn: 1.0543922	test: 1.0543920	best: 1.0543920 (600)	total: 17.1s	remaining: 5.66s
799:	learn: 1.0403101	test: 1.0403100	best: 1.0403100 (799)	total: 22.9s	remaining: 0us
bestTest = 1.040310002
bestIteration = 799
Shrink model to first 800 iterations.
inner loop 4
0:	learn: 1.3809007	test: 1.3809007	best: 1.3809007 (0)	total: 35.3ms	remaining: 28.2s
300:	learn: 1.0863706	test: 1.0863706	best: 1.0863706 (300)	total: 9.14s	remaining: 15.2s
600:	learn: 1.0531741	test: 1.0531738	best: 1.0531738 (600)	total: 17.5s	remaining: 5.8s
799:	learn: 1.0381297	test: 1.0381298	best: 1.0381298 (799)	total: 23.1s	remaining: 0us
bestTest = 1.038129763
bestIteration = 799
Shrink model to first 800 iterations.
logloss 1.0912062207561

In [23]:
sub = test[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
sub.to_csv("submission.csv",index=False)